In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import spacy
from datetime import datetime as dt
import pytz
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
merged_Tweets_df = pd.read_csv('/home/is/shuntaro-o/dev/disaster_analysis_Twitter/data/Hurricane/merged_data.csv')

In [ ]:
estimated_df = pd.read_csv('/home/is/shuntaro-o/dev/disaster_analysis_Twitter/data/Hurricane/estimated_test.csv')

In [ ]:
AuthorID_list = merged_Tweets_df['AuthorID'].unique()

In [ ]:
merged_Tweets_df["CreateTime"]

In [ ]:
time_list =  merged_Tweets_df['CreateTime'].unique()

In [ ]:
y_list = []
m_list = []
d_list = []
ymd_list = []
for time in time_list:
    y_list.append(int(time[0:4]))
    m_list.append(int(time[5:7]))
    d_list.append(int(time[8:10]))
    ymd_list.append(time[0:10])


ユーザーごと

In [ ]:
user_sequence = merged_Tweets_df[merged_Tweets_df['AuthorID']==AuthorID_list[10]]
m = folium.Map(location=[35.6, 139.7], zoom_start=7.0)
for index, r in user_sequence.iterrows():
    folium.Marker([r.UserPlase_latitude,r.UserPlase_longitude]).add_to(m)
m

推論分析

In [ ]:
estimated_True = estimated_df[estimated_df["code"]==estimated_df["geo_code"]]
estimated_False = estimated_df[estimated_df["code"]!=estimated_df["geo_code"]]

In [ ]:
m = folium.Map(location=[35.6, 139.7], zoom_start=7.0)
marker_cluster = MarkerCluster().add_to(m)
for index, r in estimated_True.iterrows():
    folium.Marker([r.UserPlase_latitude,r.UserPlase_longitude]).add_to(marker_cluster)
m

In [ ]:
m = folium.Map(location=[35.6, 139.7], zoom_start=7.0)
marker_cluster = MarkerCluster().add_to(m)
for index, r in estimated_False.iterrows():
    folium.Marker([r.UserPlase_latitude,r.UserPlase_longitude]).add_to(marker_cluster)
m

In [ ]:

# モデルのロード
nlp = spacy.load("ja_core_news_md")
# 解析対象のテキストa
input_text = "8月に東京に行く"
# モデルに解析対象のテキストを渡す
doc = nlp(input_text)
# 固有表現を抽出
for ent in doc.ents:
    print(ent.text, ent.label_, ent.start_char, ent.end_char)

In [ ]:
sentences_True = estimated_True.Text.values

In [ ]:
counter_in_True=0
for text in sentences_True:
    Flag=0
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_=="GPE":
            Flag=1
        if Flag==1:
            counter_in_True+=1
counter_in_True

In [ ]:
sentences_False = estimated_False.Text.values

In [ ]:
counter_in_False=0
for text in sentences_True:
    Flag=0
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_=="GPE":
            Flag=1
        if Flag==1:
            counter_in_False+=1
counter_in_False

In [ ]:
counter_in_False

時間分析

１dayごと

In [ ]:
time_line_df = pd.read_csv("/home/is/shuntaro-o/dev/disaster_analysis_Twitter/data/Hurricane/split_time.csv")

In [ ]:
compare_times = time_line_df.ymd.unique()

In [ ]:
compare_times.sort()

In [ ]:
tweet_nums_per_day = []
for i in compare_times:
    tweet_nums_per_day.append(len(time_line_df[time_line_df['ymd']==i]))

In [ ]:
hour_for_xlabel = []
for i in compare_times:
    hour_for_xlabel.append(str(i)[-2:])

In [ ]:
xlabel = []
for i in compare_times:
    xlabel.append('{}/{}'.format(str(i)[4:6],str(i)[6:8]))


In [ ]:
len(xlabel)

In [ ]:
plt.rcParams["figure.figsize"] = (16, 12)
plt.xticks(rotation=30)
plt.tick_params(direction = "inout", length = 5, colors = "white")
plt.xticks(np.arange(0, 123, 10))
plt.plot(xlabel,tweet_nums_per_day)
plt.xlabel("time",color="white")
plt.ylabel("sum_tweets",color="white")

1hourごと
hagibis

In [ ]:
compare_times_hour = time_line_df.ymdh.unique()
compare_times_hour.sort()
compare_times_hour = compare_times_hour[2328:2592]

In [ ]:
xlabel_hour = []
for i in compare_times_hour:
    xlabel_hour.append('{}/{}/{}'.format(str(i)[4:6],str(i)[6:8],str(i)[8:10]))

In [ ]:
tweet_nums_per_hour = []
for i in compare_times_hour:
    tweet_nums_per_hour.append(len(time_line_df[time_line_df['ymdh']==i]))

In [ ]:
from turtle import color


plt.rcParams["figure.figsize"] = (16, 12)
plt.xticks(rotation=30)
plt.tick_params(direction = "inout", length = 5, colors = "white")
plt.xticks(np.arange(0, 264, 24))
plt.plot(xlabel_hour,tweet_nums_per_hour)
plt.xlabel("time",color="white")
plt.ylabel("sum_tweets",color="white")

In [ ]:
m = folium.Map(location=[35.6, 139.7], zoom_start=5.0)
marker_cluster = MarkerCluster().add_to(m)
for index, r in time_line_df[(time_line_df['month']==10) & (time_line_df['day']==6)].iterrows():
    folium.Marker([r.UserPlase_latitude,r.UserPlase_longitude]).add_to(marker_cluster)
m